In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper

def record_and_transcribe(duration=1, sample_rate=48000, output_file='output.wav', language="en"):
    # บันทึกเสียง
    print("เริ่มบันทึกเสียง...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # รอให้การบันทึกเสร็จสิ้น
    print("บันทึกเสียงเสร็จสิ้น")
    
    # บันทึกเสียงเป็นไฟล์ .wav
    write(output_file, sample_rate, audio_data)
    print(f"ไฟล์เสียงถูกบันทึกเป็น '{output_file}'")
    
    # โหลดโมเดล Whisper
    model = whisper.load_model("medium")
    
    # ถอดเสียงจากไฟล์ที่บันทึก
    print("เริ่มถอดเสียง...")
    transcription = model.transcribe(output_file, language=language)
    
    # แสดงผลลัพธ์การถอดเสียง
    print("ข้อความที่ถอดเสียงได้:")
    print(transcription["text"])
    
# เรียกใช้งานฟังก์ชัน
record_and_transcribe()


เริ่มบันทึกเสียง...
บันทึกเสียงเสร็จสิ้น
ไฟล์เสียงถูกบันทึกเป็น 'output.wav'


 82%|██████████████████████████████▉       | 1.16G/1.42G [22:48<06:30, 721kiB/s]